In [9]:
%%bash
cd ./mt19937
python ./setup.py develop

Compiling ./srs/mt19937.pyx because it depends on srs/bounded_integers.pxi.
[1/1] Cythonizing ./srs/mt19937.pyx
running develop
running egg_info
writing srs.egg-info/PKG-INFO
writing dependency_links to srs.egg-info/dependency_links.txt
writing top-level names to srs.egg-info/top_level.txt
reading manifest file 'srs.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'srs.egg-info/SOURCES.txt'
running build_ext
building 'srs.mt19937' extension
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -DRS_RANDOMKIT=1 -I./srs -I/opt/conda/lib/python3.6/site-packages/numpy/core/include -I./srs/src/random-kit -I/opt/conda/include/python3.6m -c ./srs/mt19937.c -o build/temp.linux-x86_64-3.6/./srs/mt19937.o -std=c99 -msse2
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -DRS_RANDOMKIT=1 -I./srs -I/opt/conda/lib/python3.6/site-packages/numpy/core/include -I./srs

In file included from /opt/conda/lib/python3.6/site-packages/numpy/core/include/numpy/ndarraytypes.h:1788:0,
                 from /opt/conda/lib/python3.6/site-packages/numpy/core/include/numpy/ndarrayobject.h:18,
                 from /opt/conda/lib/python3.6/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from ./srs/mt19937.c:465:
/opt/conda/lib/python3.6/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:15:2: warning: #warning "Using deprecated NumPy API, disable it by " "#defining NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
 #warning "Using deprecated NumPy API, disable it by " \
  ^
./srs/mt19937.c:15130:18: warning: ‘__pyx_f_3srs_7mt19937_float_fill_from_double’ defined but not used [-Wunused-function]
 static PyObject *__pyx_f_3srs_7mt19937_float_fill_from_double(aug_state *__pyx_v_state, void *__pyx_v_func, PyObject *__pyx_v_size, PyObject *__pyx_v_lock) {
                  ^
./srs/distributions.c: In function ‘gauss_zig_julia’:
./

In [10]:
%%bash
cd cython_mcmc
python ./setup.py develop

See http://cython.readthedocs.io/en/latest/src/userguide/sharing_declarations.html for sharing declarations among Cython files.
running develop
running egg_info
writing cython_mcmc.egg-info/PKG-INFO
writing dependency_links to cython_mcmc.egg-info/dependency_links.txt
writing top-level names to cython_mcmc.egg-info/top_level.txt
reading manifest file 'cython_mcmc.egg-info/SOURCES.txt'
writing manifest file 'cython_mcmc.egg-info/SOURCES.txt'
running build_ext
building 'cython_mcmc.mcmc' extension
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/opt/conda/lib/python3.6/site-packages/numpy/core/include -I../mt19937/srs -I/opt/conda/include/python3.6m -c cython_mcmc/mcmc.c -o build/temp.linux-x86_64-3.6/cython_mcmc/mcmc.o
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/opt/conda/lib/python3.6/site-packages/numpy/core/include -I../mt19937/srs -I/opt/conda/include/python3.6m -c 

In file included from /opt/conda/lib/python3.6/site-packages/numpy/core/include/numpy/ndarraytypes.h:1788:0,
                 from /opt/conda/lib/python3.6/site-packages/numpy/core/include/numpy/ndarrayobject.h:18,
                 from /opt/conda/lib/python3.6/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from cython_mcmc/mcmc.c:453:
/opt/conda/lib/python3.6/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:15:2: warning: #warning "Using deprecated NumPy API, disable it by " "#defining NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
 #warning "Using deprecated NumPy API, disable it by " \
  ^
cython_mcmc/mcmc.c:2024:15: warning: ‘__pyx_f_11cython_mcmc_4mcmc_sample_norm’ defined but not used [-Wunused-function]
 static double __pyx_f_11cython_mcmc_4mcmc_sample_norm(double __pyx_v_mu, double __pyx_v_sigma) {
               ^
In file included from /opt/conda/lib/python3.6/site-packages/numpy/core/include/numpy/ndarraytypes.h:1788:0,
           

In [2]:
from cython_mcmc import mcmc

In [4]:
import numpy as np

In [8]:
np.random.seed(123)
data = np.random.randn(20)

In [9]:
%%timeit
mcmc.log_sampler_cy_v3(data, samples=15000, mu_init=1.0)

6.28 ms ± 143 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
